# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] 1452789490 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /Users/administrator/.graphlab/anaconda/lib/python2.7/site-packages/certifi/cacert.pem
1452789490 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to edwinfcampos@aol.com and will expire on December 04, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-58384 - Server binary: /Users/administrator/.graphlab/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1452789490.log
[INFO] GraphLab Server Version: 1.7.1


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 1.493168     | 6962915.603493     | 426631.749026 |
PROGRESS: | 2         | 3        | 0.000002  | 1.544067     | 6843144.200219     | 392488.929838 |
PROGRESS: | 3         | 4      

Find what features had non-zero weight.

In [6]:
model_coeff_lasso = model_all.coefficients['value']
print model_coeff_lasso

[274873.0559504957, 0.0, 0.0, 8468.531086910072, 24.420720982445214, 350.0605533860648, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 842.0680348976282, 20.024722417091304, 0.0, 0.0, 0.0]


In [7]:
import numpy as np
model_features_lasso = np.hstack([['1'],all_features])
print model_features_lasso

['1' 'bedrooms' 'bedrooms_square' 'bathrooms' 'sqft_living'
 'sqft_living_sqrt' 'sqft_lot' 'sqft_lot_sqrt' 'floors' 'floors_square'
 'waterfront' 'view' 'condition' 'grade' 'sqft_above' 'sqft_basement'
 'yr_built' 'yr_renovated']


In [8]:
print 'number of nonzero coefficients = %d' % (model_all.coefficients['value']).nnz()
deg = len(model_all.coefficients['value'])-1
print '... in a model with ' + str(deg) + ' features'

number of nonzero coefficients = 6
... in a model with 17 features


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

In [10]:
# Recall that model_coeff_lasso[0] corresponds to the constant_one = model_features_lasso[0] 
# Recall that model_coeff_lasso[1] corresponds to all_features[0] = model_features_lasso[1] 
for i in xrange(len(model_coeff_lasso)):
    if model_coeff_lasso[i] != 0.0:
        print i, model_features_lasso[i]

0 1
3 bathrooms
4 sqft_living
5 sqft_living_sqrt
13 grade
14 sqft_above


# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [11]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

In [12]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

In [13]:
(testing_feature_matrix, testing_output) = get_numpy_data(testing, all_features, 'price')
(training_feature_matrix, train_output) = get_numpy_data(training, all_features, 'price')
(validation_feature_matrix, validation_output) = get_numpy_data(validation, all_features, 'price')

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [14]:
import numpy as np
l1_penalty_values = np.logspace(1, 7, num=13)
#RSS_validation = np.empty(len(l1_penalty_values))
RSS_validation = dict()

In [15]:
for l1_penalty in l1_penalty_values:
    # Fit a regression model for a given l1 penalty on train data
    model_weights_training_l1 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    
    # Compute the RSS for a given l1 penalty on validation data
    # Recall that .predict uses an SFrame as input
    validation_model_weights_predictions = model_weights_training_l1.predict(validation)   # This inputs SFrames
    #validation_model_weights_predictions = np.dot(validation_feature_matrix, model_weights_training_l1)  # This inputs numpy arrays/matrices
    # Compute the residuals/errors
    residuals = validation_output - validation_model_weights_predictions
    # Then square and add them up
    residuals_square = residuals * residuals
    RSS = residuals_square.sum()
    RSS_validation.update({l1_penalty:RSS})
    print 'RSS on VALIDATION data, l1 =',RSS, l1_penalty

RSS on VALIDATION data, l1 = 6.25766285142e+14 10.0
RSS on VALIDATION data, l1 = 6.25766285362e+14 31.6227766017
RSS on VALIDATION data, l1 = 6.25766286058e+14 100.0
RSS on VALIDATION data, l1 = 6.25766288257e+14 316.227766017
RSS on VALIDATION data, l1 = 6.25766295212e+14 1000.0
RSS on VALIDATION data, l1 = 6.25766317206e+14 3162.27766017
RSS on VALIDATION data, l1 = 6.25766386761e+14 10000.0
RSS on VALIDATION data, l1 = 6.25766606749e+14 31622.7766017
RSS on VALIDATION data, l1 = 6.25767302792e+14 100000.0
RSS on VALIDATION data, l1 = 6.25769507644e+14 316227.766017
RSS on VALIDATION data, l1 = 6.25776517727e+14 1000000.0
RSS on VALIDATION data, l1 = 6.25799062845e+14 3162277.66017
RSS on VALIDATION data, l1 = 6.25883719085e+14 10000000.0


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [16]:
optimal_l1_penalty = min(RSS_validation, key=lambda k: RSS_validation[k])
print 'The lowest RSS using optimal L1 penalty and validation data corresponds to ...' 
print 'L1 penalty', optimal_l1_penalty
print 'and RSS', RSS_validation[optimal_l1_penalty]

The lowest RSS using optimal L1 penalty and validation data corresponds to ...
L1 penalty 10.0
and RSS 6.25766285142e+14


In [17]:
#optimal_RSS_validation = min(RSS_validation)  # This returns the smallest dictionary in key
#print 'The lowest RSS using optimal L1 penalty and validation data corresponds to RSS', RSS_validation[optimal_RSS_validation]

In [18]:
# Compute the RSS for the best l1 penalty on TEST data
# Fit a regression model for a given l1 penalty on train data
model_weights_training_bestl1 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=optimal_l1_penalty)   
# Recall that .predict uses an SFrame as input
test_model_weights_predictions = model_weights_training_bestl1.predict(testing)   # This inputs SFrames
# Compute the residuals/errors
residuals = testing_output - test_model_weights_predictions
# Then square and add them up
residuals_square = residuals * residuals
RSS_test = residuals_square.sum()
print 'RSS on TEST data, best L1 penalty=',RSS,',', optimal_l1_penalty

RSS on TEST data, best L1 penalty= 6.25883719085e+14 , 10.0


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [19]:
print 'number of nonzero coefficients = %d' % (model_weights_training_bestl1.coefficients['value']).nnz()
deg_bestL1 = len(model_weights_training_bestl1.coefficients['value'])-1
print '... in a model with ' + str(deg_bestL1) + ' features'

number of nonzero coefficients = 18
... in a model with 17 features


In [20]:
#print model_weights_training_bestl1.coefficients['value']

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [21]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [22]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [23]:
numberofnonzeros = []  # Empty list
for l1_penalty in l1_penalty_values:
    # Fit a regression model for a given l1 penalty on train data
    model_weights_training_l1 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    n_nonzeros_inmodel = (model_weights_training_l1.coefficients['value']).nnz()
    numberofnonzeros.append(n_nonzeros_inmodel)
    print 'number of nonzero coefficients = %d' % n_nonzeros_inmodel    
    #model_weights_training_l1['coefficients']['value']

number of nonzero coefficients = 18
number of nonzero coefficients = 18
number of nonzero coefficients = 18
number of nonzero coefficients = 18
number of nonzero coefficients = 17
number of nonzero coefficients = 17
number of nonzero coefficients = 17
number of nonzero coefficients = 17
number of nonzero coefficients = 17
number of nonzero coefficients = 16
number of nonzero coefficients = 15
number of nonzero coefficients = 15
number of nonzero coefficients = 13
number of nonzero coefficients = 12
number of nonzero coefficients = 10
number of nonzero coefficients = 6
number of nonzero coefficients = 5
number of nonzero coefficients = 3
number of nonzero coefficients = 1
number of nonzero coefficients = 1


In [24]:
print numberofnonzeros

[18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 15, 15, 13, 12, 10, 6, 5, 3, 1, 1]


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [25]:
#print numberofnonzeros, max_nonzeros
# Recall that numberofnonzeros is a list
nzs_nparray = np.array(numberofnonzeros)
# index in numberofnonzeros where numberofnonzeros > max_nonzero
nzs_boolean = np.where(nzs_nparray > max_nonzeros, True,False)
#print nzs_boolean
l1_penalty_min =  max(l1_penalty_values[nzs_boolean]) 
print " l1_penalty_min =", l1_penalty_min

 l1_penalty_min = 2976351441.63


In [26]:
# index in numberofnonzeros where numberofnonzeros < max_nonzero
nzs_boolean = np.where(nzs_nparray < max_nonzeros, True,False)
#print nzs_boolean
l1_penalty_max = min(l1_penalty_values[nzs_boolean])
#print l1_penalty_values
print " l1_penalty_max = ",l1_penalty_max

 l1_penalty_max =  3792690190.73


***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [27]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [28]:
#model_coeff_matrix = np.empty(len(all_features)) # Empty vector
#numberofnonzeros = []  # Empty list
#RSS_validation = dict() # Empty dictionary
count = 0  # Number of models with feature sparsity = max_nonzero 
for l1_penalty in l1_penalty_values:
    # Fit a regression model for a given l1 penalty on train data
    model_weights_training_l1 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    
    # Identify the model coefficients
    model_coeff_l1 = model_weights_training_l1.coefficients['value']    
    # Compute the number of non-zero coefficients
    n_nonzeros_inmodel = (model_coeff_l1).nnz()    
    #numberofnonzeros.append(n_nonzeros_inmodel)
    
    # Filter in the models that have sparsity equal to max_nonzero
    if n_nonzeros_inmodel == max_nonzeros:         
        # Compute the RSS for a given l1 penalty on validation dataset
        # Recall that .predict uses an SFrame as input
        validation_model_weights_predictions = model_weights_training_l1.predict(validation)   # This inputs SFrames
        #print model_coeff_l1
        #validation_model_weights_predictions = np.dot(validation_feature_matrix, model_weights_training_l1)  # This inputs numpy arrays/matrices
        # Compute the residuals/errors
        residuals = validation_output - validation_model_weights_predictions
        # Then square and add them up
        residuals_square = residuals * residuals
        RSS = residuals_square.sum()
        #RSS_validation.update({l1_penalty:RSS})
        print 'RSS_validation, L1, nnz =',RSS, l1_penalty, n_nonzeros_inmodel
        
        # Store the relevant information
        # model coefficients_matrix = Matrix[valid_l1_index,model_feature_index]
        # L1_vector = vector[valid_l1_index]
        # RSS_vector = vector[valid_l1_index]
        if count == 0:
            model_coeff_matrix = model_coeff_l1
            L1_vector = l1_penalty
            RSS_vector = RSS
        else:
            model_coeff_matrix = np.vstack([model_coeff_matrix,model_coeff_l1])
            L1_vector = np.hstack([L1_vector, l1_penalty])
            RSS_vector = np.hstack([RSS_vector, RSS])
        count += 1

RSS_validation, L1, nnz = 1.04693748875e+15 3448968612.16 7
RSS_validation, L1, nnz = 1.05114762561e+15 3491933809.48 7
RSS_validation, L1, nnz = 1.05599273534e+15 3534899006.81 7
RSS_validation, L1, nnz = 1.06079953176e+15 3577864204.13 7


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [29]:
boolean_indexes = np.where(RSS_vector == min(RSS_vector), True,False)
print 'The lowest RSS using optimal L1 penalty and validation data corresponds to ...' 
print 'L1 penalty', L1_vector[boolean_indexes]
print 'and RSS', RSS_vector[boolean_indexes]

The lowest RSS using optimal L1 penalty and validation data corresponds to ...
L1 penalty [  3.44896861e+09]
and RSS [  1.04693749e+15]


In [30]:
model_coeff_optimal_l1 = model_coeff_matrix[boolean_indexes,:]  # This gives a 2D array of np.shape = (1, 18) 
model_coeff_optimal_l1 = model_coeff_optimal_l1[0,:]  # This gives a 1D array of np.shape = 18
print model_coeff_optimal_l1

[  2.22253193e+05   6.61722718e+02   0.00000000e+00   1.58739573e+04
   3.24102215e+01   6.90114773e+02   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   2.89942027e+03   3.00115753e+01   0.00000000e+00
   0.00000000e+00   0.00000000e+00]


In [33]:
print all_features

['bedrooms', 'bedrooms_square', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'sqft_lot', 'sqft_lot_sqrt', 'floors', 'floors_square', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']


In [31]:
model_features_optimal_l1 = np.hstack([['1'],all_features])
print model_features_optimal_l1

['1' 'bedrooms' 'bedrooms_square' 'bathrooms' 'sqft_living'
 'sqft_living_sqrt' 'sqft_lot' 'sqft_lot_sqrt' 'floors' 'floors_square'
 'waterfront' 'view' 'condition' 'grade' 'sqft_above' 'sqft_basement'
 'yr_built' 'yr_renovated']


In [32]:
# Recall that model_coeff_optimal_l1[0] corresponds to the constant_one = model_features_optimal_l1[0] 
# Recall that model_coeff_optimal_l1[1] corresponds to all_features[0] = model_features_optimal_l1[1] 
for i in xrange(len(model_coeff_optimal_l1)):
    if model_coeff_optimal_l1[i] != 0.0:
        print i, model_features_optimal_l1[i]
        
#for i in xrange(len(all_features)):
#    if model_coeff_optimal_l1[i] != 0.0:
#        print i, all_features[i]

0 1
1 bedrooms
3 bathrooms
4 sqft_living
5 sqft_living_sqrt
13 grade
14 sqft_above
